In [56]:
from flask import Flask,jsonify,request

In [57]:
app = Flask(__name__)

In [58]:
# import all required libraries in this section
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib
import pickle
import numpy as np

In [52]:
dd= pd.read_excel('FalseAlarmCases.xlsx')

In [43]:
dd.head()

,Case No.,Ambient Temperature( deg C),Calibration(days),Unwanted substance deposition(0/1),Humidity(%),H2S Content(ppm),detected by(% of sensors),Spuriosity Index(0/1),Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Case # 1,-2,226,1,96,9,21,1,NaN,NaN,NaN
1,Case # 2,4,134,1,83,4,77,0,NaN,NaN,NaN
2,Case # 3,7,163,0,69,2,81,0,NaN,NaN,NaN
3,Case # 4,5,162,0,80,6,69,0,NaN,NaN,NaN
4,Case # 5,2,192,1,87,3,67,0,NaN,NaN,NaN


In [44]:
df = dd.iloc[:,1:8]
df

,Ambient Temperature( deg C),Calibration(days),Unwanted substance deposition(0/1),Humidity(%),H2S Content(ppm),detected by(% of sensors),Spuriosity Index(0/1)
0,-2,226,1,96,9,21,1
1,4,134,1,83,4,77,0
2,7,163,0,69,2,81,0
3,5,162,0,80,6,69,0
4,2,192,1,87,3,67,0
...,...,...,...,...,...,...,...
1887,6,195,1,72,5,79,0
1888,8,134,1,94,9,22,1
1889,1,32,0,95,4,100,0
1890,6,31,0,93,6,39,1


In [59]:
# change the path in pd.read_excel method before you run 
@app.route('/train_my_model')
def train():
    # load the data file
    data = pd.read_excel('FalseAlarmCases.xlsx')
    # remove columns which we don't want to use
    df = data.iloc[:,1:8]
    # spilt data in x and y axis variables
    x = df.iloc[:,0:6]
    y = df['Spuriosity Index(0/1)']
    # create object of GaussianNB class
    nb = GaussianNB()
    # rain the model
    nb.fit(x,y)
    joblib.dump(nb,'train_data.pkl')
    
    return 'Model trained successfully....'
    

In [60]:
@app.route('/test_my_model' , methods = ['POST'])
def test():
    train_data_file = joblib.load('train_data.pkl')
    test_data = request.get_json()
    at = test_data['Ambient Temperature( deg C)']
    ca = test_data['Calibration(days)']
    un = test_data['Unwanted substance deposition(0/1)']
    humidity = test_data['Humidity(%)']
    h2s = test_data['H2S Content(ppm)']
    dt = test_data['detected by(% of sensors)']
    array_test = np.array([at,ca,un,humidity,h2s,dt])
    tarray = array_test.reshape(1,6)
    df_test = pd.DataFrame(tarray, columns= ['Ambient Temperature( deg C)', 'Calibration(days)',
       'Unwanted substance deposition(0/1)', 'Humidity(%)', 'H2S Content(ppm)',
       'detected by(% of sensors)'])
    y_pred = train_data_file.predict(df_test)
    
    if y_pred == 0:
        return ' True Alarm, This is Dangerous'
    else:
        return 'False Alarm'

In [ ]:
app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Dec/2020 17:50:02] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Dec/2020 17:50:53] "POST /test_model HTTP/1.1" 404 -
